In [1]:
import pandas as pd
import numpy as np

import torch
import pathlib
import random
import cv2
import gc
import sys
import matplotlib.pyplot as plt
from collections import Counter

import solt as sl
import solt.transforms as slt
import sklearn.model_selection as ms
import torch.nn.functional as F

from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score
from torch import nn
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torchvision.models as models
import torch.optim as optim

import torch
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import json
import time
import requests
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel
import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device}')

Using cpu


In [5]:
base_path = "C:/Users/anush/OneDrive/Documents/Sem3/AI in Health Technology/Project/"

In [10]:
criterion = nn.CrossEntropyLoss()

In [12]:
def validate(model, testloader, criterion):
    model.eval()
    print('Validation')
    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    list_preds = []
    list_labels = []
    with torch.no_grad():
        for i, data in tqdm(enumerate(testloader), total=len(testloader)):
            counter += 1
            
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            # Forward pass.
            outputs = model(image)
            # Calculate the loss.
            loss = criterion(outputs, labels)
            valid_running_loss += loss.item()
            # Calculate the accuracy.
            _, preds = torch.max(outputs.data, 1)
            valid_running_correct += (preds == labels).sum().item()
            list_preds.append(preds)
            list_labels.append(labels)
            
            
    
    # Loss and accuracy for the complete epoch.
    epoch_loss = valid_running_loss / counter
    epoch_acc = 100. * (valid_running_correct / len(testloader.dataset))
    
    #for balanced accuracy
    list_true = []
    for i in range(len(list_labels)):
        for j in range(0, list_labels[i].shape[0]):
            list_true.append(int(list_labels[i][j]))

    list_pred = []
    for i in range(len(list_preds)):
        for j in range(0, list_preds[i].shape[0]):
            list_pred.append(int(list_preds[i][j]))
            
    balanced_acc = balanced_accuracy_score(list_true, list_pred)
    return epoch_loss, epoch_acc, balanced_acc, list_true, list_pred

### Model 1: DataSubset + EfficientNet_b4
Loading model with highest Validation BACC: epoch 38 -> model_checkpoint_37

In [32]:
def get_train_transform_1(pretrained=True):
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.3, contrast=0.3),
        transforms.Resize((380, 380)),
        transforms.ToTensor(),
        normalize_transform()
    ])
    return train_transform


# Validation transforms
def get_valid_transform_1(pretrained=True):
    valid_transform = transforms.Compose([
        transforms.Resize((380, 380)),
        transforms.ToTensor(),
        normalize_transform()
    ])
    return valid_transform

def get_datasets_1(pretrained=True):
    dataset_test_valTF = datasets.ImageFolder(
        base_path+"/data/test", 
        transform=(get_valid_transform_1(pretrained=True))
    )
    
    dataset_test_trainTF = datasets.ImageFolder(
        base_path+"/data/test", 
        transform=(get_train_transform_1(pretrained=True))
    )
    return dataset_test_valTF, dataset_test_trainTF


def get_data_loaders_1(dataset_test_valTF, dataset_test_trainTF, BATCH_SIZE=8):
    test_loader_valTF  = DataLoader(dataset_test_valTF, batch_size=BATCH_SIZE, shuffle=False)
    test_loader_trainTF  = DataLoader(dataset_test_trainTF, batch_size=BATCH_SIZE, shuffle=False)
    return test_loader_valTF, test_loader_trainTF

In [33]:
dataset_test_valTF_1, dataset_test_trainTF_1 = get_datasets_1(True)
test_loader_valTF_1, test_loader_trainTF_1 = get_data_loaders_1(dataset_test_valTF_1, dataset_test_trainTF_1)

In [34]:
PATH_1 = base_path+f"/efficientnet_b4_dup_subsetData/model_checkpoint_{37}.pt"
model_1 = torch.load(PATH_1, weights_only=False)

In [48]:
total_params_1 = sum(p.numel() for p in model_1.parameters())
print(f"{total_params_1:,} total parameters.")
total_trainable_params_1 = sum(
    p.numel() for p in model_1.parameters() if p.requires_grad)
print(f"{total_trainable_params_1:,} training parameters.")

18,008,909 total parameters.
460,293 training parameters.


In [35]:
loss_1a, accuracy_1a, balanced_acc_1a, list_true_1a, list_pred_1a = validate(model_1, test_loader_valTF_1, criterion)

Validation


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [02:51<00:00,  3.72s/it]


In [36]:
loss_1b, accuracy_1b, balanced_acc_1b, list_true_1b, list_pred_1b = validate(model_1, test_loader_trainTF_1, criterion)

Validation


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [04:03<00:00,  5.28s/it]


In [74]:
cks_1a = cohen_kappa_score(list_true_1a, list_pred_1a)
cks_1b = cohen_kappa_score(list_true_1b, list_pred_1b)

In [75]:
print(f"With Only Resizing of images\n Test loss: {loss_1a:.3f}\n Test acc: {accuracy_1a:.3f}\n Test BACC: {balanced_acc_1a:.3f}\n Cohen-Kappa: {cks_1a:.3f}")
print(f"With Training Image preprocessing\n Test loss: {loss_1b:.3f}\n Test acc: {accuracy_1b:.3f}\n Test BACC: {balanced_acc_1b:.3f}\n Cohen-Kappa: {cks_1b:.3f}")

With Only Resizing of images
 Test loss: 1.444
 Test acc: 30.601
 Test BACC: 0.272
 Cohen-Kappa: 0.105
With Training Image preprocessing
 Test loss: 1.456
 Test acc: 30.328
 Test BACC: 0.272
 Cohen-Kappa: 0.107


### Model 2: Baseline 1
##### EfficientNet_b0 + Linear Classification Layer
Loading model with highest Validation BACC: epoch 17 -> model_checkpoint_16

In [26]:
def get_train_transform_2(pretrained=True):
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        normalize_transform()
    ])
    return train_transform


# Validation transforms
def get_valid_transform_2(pretrained=True):
    valid_transform = transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        normalize_transform()
    ])
    return valid_transform

def get_datasets_2(pretrained=True):
    dataset_test_valTF = datasets.ImageFolder(
        base_path+"/data/test", 
        transform=(get_valid_transform_2(pretrained=True))
    )
    
    dataset_test_trainTF = datasets.ImageFolder(
        base_path+"/data/test", 
        transform=(get_train_transform_2(pretrained=True))
    )
    return dataset_test_valTF, dataset_test_trainTF


def get_data_loaders_2(dataset_test_valTF, dataset_test_trainTF, BATCH_SIZE=8):
    test_loader_valTF  = DataLoader(dataset_test_valTF, batch_size=BATCH_SIZE, shuffle=False)
    test_loader_trainTF  = DataLoader(dataset_test_trainTF, batch_size=BATCH_SIZE, shuffle=False)
    return test_loader_valTF, test_loader_trainTF

In [27]:
dataset_test_valTF_2, dataset_test_trainTF_2 = get_datasets_2(True)
test_loader_valTF_2, test_loader_trainTF_2 = get_data_loaders_2(dataset_test_valTF_2, dataset_test_trainTF_2)

In [28]:
PATH_2 = base_path+f"/baseline_1/model_checkpoint_{16}.pt"
model_2 = torch.load(PATH_2, weights_only=False)

In [49]:
total_params_2 = sum(p.numel() for p in model_2.parameters())
print(f"{total_params_2:,} total parameters.")
total_trainable_params_2 = sum(
    p.numel() for p in model_2.parameters() if p.requires_grad)
print(f"{total_trainable_params_2:,} training parameters.")

4,013,953 total parameters.
6,405 training parameters.


In [29]:
loss_2a, accuracy_2a, balanced_acc_2a, list_true_2a, list_pred_2a = validate(model_2, test_loader_valTF_2, criterion)

Validation


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [01:12<00:00,  1.58s/it]


In [30]:
loss_2b, accuracy_2b, balanced_acc_2b, list_true_2b, list_pred_2b = validate(model_2, test_loader_trainTF_2, criterion)

Validation


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [04:12<00:00,  5.49s/it]


In [72]:
cks_2a = cohen_kappa_score(list_true_2a, list_pred_2a)
cks_2b = cohen_kappa_score(list_true_2b, list_pred_2b)

In [73]:
print(f"With Only Resizing of images\n Test loss: {loss_2a:.3f}\n Test acc: {accuracy_2a:.3f}\n Test BACC: {balanced_acc_2a:.3f}\n Cohen-Kappa: {cks_2a:.3f}")
print(f"With Training Image preprocessing\n Test loss: {loss_2b:.3f}\n Test acc: {accuracy_2b:.3f}\n Test BACC: {balanced_acc_2b:.3f}\n Cohen-Kappa: {cks_2b:.3f}")

With Only Resizing of images
 Test loss: 0.614
 Test acc: 77.596
 Test BACC: 0.515
 Cohen-Kappa: 0.633
With Training Image preprocessing
 Test loss: 0.602
 Test acc: 78.415
 Test BACC: 0.533
 Cohen-Kappa: 0.645


### Model 3: RandomWeightSampler + EfficientNet_b4
Loading model with highest Validation BACC: epoch 18 -> model_checkpoint_17

In [81]:
def get_train_transform_3(pretrained=True):
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.3, contrast=0.3),
        transforms.Resize((380, 380)),
        transforms.ToTensor(),
        normalize_transform()
    ])
    return train_transform


def get_valid_transform_3(pretrained=True):
    valid_transform = transforms.Compose([
        transforms.Resize((380, 380)),
        transforms.ToTensor(),
        normalize_transform()
    ])
    return valid_transform


def get_datasets_3(pretrained=True):
    dataset_test_valTF = datasets.ImageFolder(
        base_path+"/data/test", 
        transform=(get_valid_transform_3(pretrained=True))
    )
    
    dataset_test_trainTF = datasets.ImageFolder(
        base_path+"/data/test", 
        transform=(get_train_transform_3(pretrained=True))
    )
    return dataset_test_valTF, dataset_test_trainTF


def get_data_loaders_3(dataset_test_valTF, dataset_test_trainTF, BATCH_SIZE=8):
    test_loader_valTF  = DataLoader(dataset_test_valTF, batch_size=BATCH_SIZE, shuffle=False)
    test_loader_trainTF  = DataLoader(dataset_test_trainTF, batch_size=BATCH_SIZE, shuffle=False)
    return test_loader_valTF, test_loader_trainTF

In [82]:
dataset_test_valTF_3, dataset_test_trainTF_3 = get_datasets_3(True)
test_loader_valTF_3, test_loader_trainTF_3 = get_data_loaders_3(dataset_test_valTF_3, dataset_test_trainTF_3)

In [83]:
PATH_3 = base_path+f"/efficientnet_b4_randomweight/model_checkpoint_{17}.pt"
model_3 = torch.load(PATH_3, weights_only=False)

In [84]:
total_params_3 = sum(p.numel() for p in model_3.parameters())
print(f"{total_params_3:,} total parameters.")
total_trainable_params_3 = sum(
    p.numel() for p in model_3.parameters() if p.requires_grad)
print(f"{total_trainable_params_3:,} training parameters.")

18,008,909 total parameters.
460,293 training parameters.


In [85]:
loss_3a, accuracy_3a, balanced_acc_3a, list_true_3a, list_pred_3a = validate(model_3, test_loader_valTF_3, criterion)
loss_3b, accuracy_3b, balanced_acc_3b, list_true_3b, list_pred_3b = validate(model_3, test_loader_trainTF_3, criterion)

Validation


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [02:45<00:00,  3.60s/it]


Validation


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [03:48<00:00,  4.97s/it]


In [86]:
cks_3a = cohen_kappa_score(list_true_3a, list_pred_3a)
cks_3b = cohen_kappa_score(list_true_3b, list_pred_3b)

In [87]:
print(f"With Only Resizing of images\n Test loss: {loss_3a:.3f}\n Test acc: {accuracy_3a:.3f}\n Test BACC: {balanced_acc_3a:.3f}\n Kappa-Cohen: {cks_3a:.3f}")
print(f"With Training Image preprocessing\n Test loss: {loss_3b:.3f}\n Test acc: {accuracy_3b:.3f}\n Test BACC: {balanced_acc_3b:.3f}\n Kappa-Cohen: {cks_3b:.3f}")

With Only Resizing of images
 Test loss: 0.583
 Test acc: 80.601
 Test BACC: 0.578
 Kappa-Cohen: 0.687
With Training Image preprocessing
 Test loss: 0.612
 Test acc: 79.235
 Test BACC: 0.549
 Kappa-Cohen: 0.664


### Model 4: Data Augmentation + EfficientNet_b4
(1) Loading model with highest Training BACC: epoch 12 -> model_checkpoint_11

(2) Loading model with highest Validation BACC: epoch 18 -> model_checkpoint_17

In [38]:
def get_train_transform_4(pretrained=True):
    train_transform = transforms.Compose([
        transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
        transforms.ColorJitter(brightness=(1.4, 1.6), contrast=(1.5,2)),
        transforms.Resize((380, 380)),
        transforms.ToTensor(),
        normalize_transform()
    ])
    return train_transform


def get_valid_transform_4(pretrained=True):
    valid_transform = transforms.Compose([
        transforms.Resize((380, 380)),
        transforms.ToTensor(),
        normalize_transform()
    ])
    return valid_transform


def get_datasets_4(pretrained=True):
    dataset_test_valTF = datasets.ImageFolder(
        base_path+"/data/test", 
        transform=(get_valid_transform_4(pretrained=True))
    )
    
    dataset_test_trainTF = datasets.ImageFolder(
        base_path+"/data/test", 
        transform=(get_train_transform_4(pretrained=True))
    )
    return dataset_test_valTF, dataset_test_trainTF


def get_data_loaders_4(dataset_test_valTF, dataset_test_trainTF, BATCH_SIZE=8):
    test_loader_valTF  = DataLoader(dataset_test_valTF, batch_size=BATCH_SIZE, shuffle=False)
    test_loader_trainTF  = DataLoader(dataset_test_trainTF, batch_size=BATCH_SIZE, shuffle=False)
    return test_loader_valTF, test_loader_trainTF

In [39]:
dataset_test_valTF_4, dataset_test_trainTF_4 = get_datasets_4(True)
test_loader_valTF_4, test_loader_trainTF_4 = get_data_loaders_4(dataset_test_valTF_4, dataset_test_trainTF_4)

In [40]:
PATH_4 = base_path+f"/efficientnet_b4_dup_augData/model_checkpoint_{11}.pt"
model_4 = torch.load(PATH_4, weights_only=False)

In [51]:
total_params_4 = sum(p.numel() for p in model_4.parameters())
print(f"{total_params_4:,} total parameters.")
total_trainable_params_4 = sum(
    p.numel() for p in model_4.parameters() if p.requires_grad)
print(f"{total_trainable_params_4:,} training parameters.")

18,008,909 total parameters.
460,293 training parameters.


In [41]:
loss_4a, accuracy_4a, balanced_acc_4a, list_true_4a, list_pred_4a = validate(model_4, test_loader_valTF_4, criterion)

Validation


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [02:48<00:00,  3.67s/it]


In [42]:
loss_4b, accuracy_4b, balanced_acc_4b, list_true_4b, list_pred_4b = validate(model_4, test_loader_trainTF_4, criterion)

Validation


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [03:56<00:00,  5.14s/it]


In [76]:
cks_4a = cohen_kappa_score(list_true_4a, list_pred_4a)
cks_4b = cohen_kappa_score(list_true_4b, list_pred_4b)

In [77]:
print(f"With Only Resizing of images\n Test loss: {loss_4a:.3f}\n Test acc: {accuracy_4a:.3f}\n Test BACC: {balanced_acc_4a:.3f}\n Cohen-Kappa: {cks_4a:.3f}")
print(f"With Training Image preprocessing\n Test loss: {loss_4b:.3f}\n Test acc: {accuracy_4b:.3f}\n Test BACC: {balanced_acc_4b:.3f}\n Cohen-Kappa: {cks_4b:.3f}")

With Only Resizing of images
 Test loss: 14.171
 Test acc: 9.016
 Test BACC: 0.200
 Cohen-Kappa: 0.000
With Training Image preprocessing
 Test loss: 13.189
 Test acc: 9.290
 Test BACC: 0.201
 Cohen-Kappa: 0.002


In [44]:
PATH_5 = base_path+f"/efficientnet_b4_dup_augData/model_checkpoint_{17}.pt"
model_5 = torch.load(PATH_5, weights_only=False)

In [52]:
total_params_5 = sum(p.numel() for p in model_5.parameters())
print(f"{total_params_5:,} total parameters.")
total_trainable_params_5 = sum(
    p.numel() for p in model_5.parameters() if p.requires_grad)
print(f"{total_trainable_params_5:,} training parameters.")

18,008,909 total parameters.
460,293 training parameters.


In [45]:
loss_5a, accuracy_5a, balanced_acc_5a, list_true_5a, list_pred_5a = validate(model_5, test_loader_valTF_4, criterion)

Validation


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [02:58<00:00,  3.88s/it]


In [46]:
loss_5b, accuracy_5b, balanced_acc_5b, list_true_5b, list_pred_5b = validate(model_5, test_loader_trainTF_4, criterion)

Validation


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [03:57<00:00,  5.17s/it]


In [78]:
cks_5a = cohen_kappa_score(list_true_5a, list_pred_5a)
cks_5b = cohen_kappa_score(list_true_5b, list_pred_5b)

In [79]:
print(f"With Only Resizing of images\n Test loss: {loss_5a:.3f}\n Test acc: {accuracy_5a:.3f}\n Test BACC: {balanced_acc_5a:.3f}\n Cohen-Kappa: {cks_5a:.3f}")
print(f"With Training Image preprocessing\n Test loss: {loss_5b:.3f}\n Test acc: {accuracy_5b:.3f}\n Test BACC: {balanced_acc_5b:.3f}\n Cohen-Kappa: {cks_5b:.3f}")

With Only Resizing of images
 Test loss: 10.200
 Test acc: 9.016
 Test BACC: 0.200
 Cohen-Kappa: 0.000
With Training Image preprocessing
 Test loss: 9.333
 Test acc: 10.929
 Test BACC: 0.207
 Cohen-Kappa: 0.012


### Hypothesis Testing

In [89]:
accuracy_1a, balanced_acc_1a, cks_1a

(30.601092896174865, 0.27199961946310325, 0.10526366444981272)

In [90]:
accuracy_3a, balanced_acc_3a, cks_3a

(80.60109289617486, 0.5783530783180146, 0.6869533791109506)

In [88]:
ttest_rel([accuracy_1a, balanced_acc_1a, cks_1a], [accuracy_3a, balanced_acc_3a, cks_3a]).pvalue

0.4124462792589929